In [232]:
using HTTP, DataFrames, JSON3, Dates

In [233]:
# Jonh Hopkins University CSSE API

function assemble_base_url(base; data_type) 
    data_type == :totals ? (url = base * "jhucsse/") : (url = base * "historical/")
    return url
end

#function assemble_url_jhucsse(base_url, data_type; countries = nothing, provinces = nothing, USstates = nothing, UScounties = nothing) 
#    
#    if data_type == :totals
#        if countries == :all 
#            url = base_url
#        elseif UScounties != nothing
#            url = base_url * "counties"  
#        else
#            error("This method only applies for assembling url for total COVID-19 data. Please remember that
#            'counties' and 'countries' are mutually exclusive")
#        end
#        
#    elseif data_type == :accumulated
#        countries == :all ? (url = base_url * "all") : error("For accumulated data type, the url can only be assembled for all the countries")
#        
#    elseif data_type == :timeseries
#        if (countries != nothing && provinces == nothing)
#            countries == :all ? (url = base_url) : (url = base_url * join(countries, ","))
#        
#        elseif provinces !=  nothing
#            if length(provinces) == 1                
#                url = base_url * string.(countries[1]) * "/" * string.(provinces[1])
#            elseif length(provinces) > 1
#                url = base_url * string.(countries[1]) * "/" * join(map(x -> replace(x, " " => "%20"), string.(provinces)), ",")
#            else
#                error("'provinces' must be a vector of 'string's of 'length > 0' ")
#            end
#        
#        elseif UScounties != nothing
#            counties_url = base_url * "usacounties"
#            if UScounties == :all
#                url = counties_url
#            elseif USstates != nothing
#                url = counties_url * "/" * string(USstates[1]) * "/"
#            else
#                error("An url can only be assembled for one county or for all the countries from one state")
#            end           
#            
#        else
#            error("A single country must be specified to generate 'provinces' time series url.")
#        end
#    
#    else
#        error("It was not possible to assemble the url as indicated by the arguments.")
#    end
#end

function assemble_url_jhucsse(base_url, data_type::Val{:totals}; countries = nothing, provinces = nothing, USstates = nothing, UScounties = nothing) 
    if countries == :all 
        url = base_url
    elseif UScounties != nothing
        url = base_url * "counties"  
    else
        error("This method only applies for assembling url for total COVID-19 data. Please remember that
        'counties' and 'countries' are mutually exclusive")
    end
end


function assemble_url_jhucsse(base_url, data_type::Val{:accumulated}; countries = nothing, provinces = nothing, USstates = nothing, UScounties = nothing) 
    countries == :all ? (url = base_url * "all") : error("For accumulated data type, the url can only be assembled for all the countries")
end


function assemble_url_jhucsse(base_url, data_type::Val{:timeseries}; countries = nothing, provinces = nothing, USstates = nothing, UScounties = nothing) 
   
    if (countries != nothing && provinces == nothing)
        countries == :all ? (url = base_url) : (url = base_url * join(countries, ","))
    
    elseif provinces !=  nothing
        if length(provinces) == 1                
            url = base_url * string.(countries[1]) * "/" * string.(provinces[1])
        elseif length(provinces) > 1
            url = base_url * string.(countries[1]) * "/" * join(map(x -> replace(x, " " => "%20"), string.(provinces)), ",")
        else
            error("'provinces' must be a vector of 'string's of 'length > 0' ")
        end
    
    elseif UScounties != nothing
        counties_url = base_url * "usacounties"
        if UScounties == :all
            url = counties_url
        elseif USstates != nothing
            url = counties_url * "/" * string(USstates[1]) * "/"
        else
            error("An url can only be assembled for one county or for all the countries from one state")
        end           
        
    else
        error("A single country must be specified to generate 'provinces' time series url.")
    end
end


function assemble_url_jhucsse(base_url, data_type::Symbol; kwargs...) 
    try
       assemble_url_jhucsse(base_url, Val(data_type); kwargs...)
    catch e
        error("It was not possible to assemble the url as indicated by the arguments.")
    end
    
end


assemble_url_jhucsse (generic function with 1 method)

In [214]:
function assemble_df_jhucsse(raw_df; countries = nothing, USstates = nothing)
    timeline = vcat(DataFrame.(raw_df[:, :timeline])...)
    data = hcat(select(raw_df, Not(:timeline)), timeline)
    date_arr = String.(keys(data.cases[1]))
    df_arr = []
    
    if countries != nothing
        for (idx, country) in enumerate(eachrow(data))
            dict = [Dict(:country => data.country[idx], :province => province, :date => date, :cases => data.cases[idx][date], 
                    :deaths => data.deaths[idx][date], :revovered => data.recovered[idx][date]) 
                    for (country, province) in eachrow(data) for date in date_arr]
            df = DataFrame(dict)
            push!(df, df_arr)
        end

    elseif USstates != nothing 
        dict = [Dict(:state => state, :county => county, :date => date, :cases => data.cases[idx][date_arr[date]], :deaths => data.deaths[idx][date_arr[date]]) 
            for (idx, (state, county)) in enumerate(eachrow(data)) for date in date_arr]
        df = DataFrame(dict)
    else

    end

    return DataFrame(df_arr)
end


#function opendisease_JHUCSSE(data_type; lastdays = nothing, countries = nothing, provinces = nothing, USstates = nothing, UScounties = nothing)
#    
#    """
#    'countries': An array of country names in iso2, iso3, or country ID code format.
#    'provinces': An array of 'Symbol'. It contains provinces names spelled correctly separated by ','.
#    """
#    
#    try
#        base_jhucsse = "https://disease.sh/v3/covid-19/"
#        base_url = assemble_base_url(base_jhucsse, data_type = data_type)
#        params = Dict("lastdays" => lastdays)
#       
#        if data_type == :totals  # Total cases are only available for counties and for all the contries
#            
#            if countries != nothing
#                countries == :all ? (url = base_url) : (url = assemble_url_jhucsse(base_url, data_type = data_type, countries = countries)) 
#                get_data = HTTP.get(url)
#                raw_df = vcat(DataFrame.(JSON3.read(get_data.body))...)
#                stats = vcat(DataFrame.(raw_df[!, :stats])...)
#                coordinates = vcat(DataFrame.(raw_df[!, :coordinates])...)
#                data = hcat(select(raw_df, [:country, :county, :province, :updatedAt]), stats, coordinates)
#                
#            elseif UScounties != nothing
#                url = assemble_url_jhucsse(base_url, data_type, UScounties = UScounties)
#                get_data =  HTTP.get(url)   #get_data = HTTP.get("https://disease.sh/v3/covid-19/jhucsse/counties")
#                raw_df = vcat(DataFrame.(JSON3.read(get_data.body))...)
#                stats = vcat(DataFrame.(raw_df[!, :stats])...)
#                coordinates = vcat(DataFrame.(raw_df[!, :coordinates])...)
#                df = hcat(select(raw_df, [:country, :county, :province, :updatedAt]), stats, coordinates)
#                if UScounties == :all 
#                    data = df 
#                elseif USstates != nothing
#                    data = filter(x -> x.province in string.(USstates), filter(x -> x.county in string.(UScounties), df))
#                else
#                    data = filter(x -> x.county in string.(UScounties), df)  #UScounty is a vector of Symbols
#                end
#                
#            else
#                error("COVID-19 Totals are only available for all the countries ('countries = :all') and for US counties 
#                    (all counties 'counties = :all', or a subset of them 'counties = (:countie1, :countie2, ...))'.")
#            end
#            
#        elseif data_type == :accumulated
#            countries == :all ? (url = assemble_url_jhucsse(base_url, data_type, countries = countries)) : error("Accumulated data is olnly available for all the countries")
#            get_data = HTTP.get(url, query = params)
#            data = DataFrame(JSON3.read(get_data.body))
#            #data = assemble_df_jhucsse(df)
#            
#        elseif data_type == :timeseries
#            if (countries != nothing && provinces == nothing)
#                if countries == :all
#                    url = assemble_url_jhucsse(base_url, data_type, countries = countries)
#                    get_data = HTTP.get(url, query = params)
#                    df  = DataFrame(JSON3.read(get_data.body))
#                    data = assemble_df_jhucsse(df, countries = countries)
#                    #timeline = vcat(DataFrame.(df[:, :timeline])...)
#                    #data = hcat(select(df, Not(:timeline)), timeline)
#                    
#                elseif countries != :all
#                    url = assemble_url_jhucsse(base_url, data_type, countries = countries)
#                    get_data = HTTP.get(url, query = params)
#                    length(countries) == 1 ? (df = DataFrame(JSON3.read(get_data.body))) : (df = vcat(DataFrame.(JSON3.read(get_data.body))...))
#                    data = assemble_df_jhucsse(df, countries = countries)
#                    #timeline = vcat(DataFrame.(df[:, :timeline])...)
#                    #data = hcat(select(df, Not(:timeline)), timeline)
#                else
#                    error("'countries' must be ':all' or a vector of 'Symbol's with the name of the countries of interest")
#                end   
#            
#            elseif provinces != nothing
#                #provinces names must not be capitalized and the words must be separated by one space. Example: "australian capital territory"
#                url = assemble_url_jhucsse(base_url, data_type, countries = countries, provinces = provinces)
#                get_data = HTTP.get(url, query = params)
#                df = DataFrame(JSON3.read(get_data.body))
#                data = assemble_df_jhucsse(df, countries = countries)
#                #timeline = vcat(DataFrame.(df[:, :timeline])...)
#                #data = hcat(select(df, Not(:timeline)), timeline)
#                #data = filter(x -> x.province in pronvinces, df)
#
#            elseif UScounties == :all
#                url = assemble_url_jhucsse(base_url, data_type, UScounties = UScounties)
#                get_data = HTTP.get(url, query = params)
#                data = DataFrame(USstate = JSON3.read(get_data.body))
#                #USstates == nothing ? data = filter(x -> x.county in string.(UScounties), df) : data = df
#            
#            elseif USstates != nothing
#                url = assemble_url_jhucsse(base_url, data_type, USstates = USstates, UScounties = UScounties)
#                get_data = HTTP.get(url, query = params)
#                raw_df = DataFrame(JSON3.read(get_data.body))
#                df = assemble_df_jhucsse(raw_df, USstates = USstates)
#                #timeline = vcat(DataFrame.(raw_df[:, :timeline])...)
#                #df = hcat(select(raw_df, Not(:timeline)), timeline)
#                UScounties != nothing ? data = filter(x -> x.county in string.(UScounties), df) : data = df
#            else
#                nothing
#            end
#                              
#        else
#            error("It was not possible to request the data as indicated. Please check if following requirements 
#                are met in your data request: \n
#                - COVID-19 Totals are only available for all the countries ('countries = :all') and for US counties 
#                (all counties 'counties = :all', or a subset of them 'counties = (:countie1, :countie2, ...))'. \n
#                - 'lastdays' only applies for time-series data, and it must be always specified \n 
#                 # for Time series data, the number of days (lastdays) must be specified
#            # lastdays only applies for time series
#            # countries must be a vector of country's IDs, iso2 or iso3")
#        end
#        
#    return data
#   
#    catch e
#        print(e)
#    finally
#        
#    end
#end

opendisease_JHUCSSE (generic function with 1 method)

In [ ]:
function opendisease_JHUCSSE(data_type::Val{:totals}; lastdays = nothing, countries = nothing, provinces = nothing, USstates = nothing, UScounties = nothing)   
    """
    'countries': An array of country names in iso2, iso3, or country ID code format.
    'provinces': An array of 'Symbol'. It contains provinces names spelled correctly separated by ','.
    """
    # Total cases are only available for counties and for all the contries
    try
        base_jhucsse = "https://disease.sh/v3/covid-19/"
        base_url = assemble_base_url(base_jhucsse, data_type = data_type)
        params = Dict("lastdays" => lastdays)
            
        if countries != nothing
            countries == :all ? (url = base_url) : (url = assemble_url_jhucsse(base_url, data_type = data_type, countries = countries)) 
            get_data = HTTP.get(url)
            raw_df = vcat(DataFrame.(JSON3.read(get_data.body))...)
            stats = vcat(DataFrame.(raw_df[!, :stats])...)
            coordinates = vcat(DataFrame.(raw_df[!, :coordinates])...)
            data = hcat(select(raw_df, [:country, :county, :province, :updatedAt]), stats, coordinates)
            
        elseif UScounties != nothing
            url = assemble_url_jhucsse(base_url, data_type, UScounties = UScounties)
            get_data =  HTTP.get(url)   #get_data = HTTP.get("https://disease.sh/v3/covid-19/jhucsse/counties")
            raw_df = vcat(DataFrame.(JSON3.read(get_data.body))...)
            stats = vcat(DataFrame.(raw_df[!, :stats])...)
            coordinates = vcat(DataFrame.(raw_df[!, :coordinates])...)
            df = hcat(select(raw_df, [:country, :county, :province, :updatedAt]), stats, coordinates)
            if UScounties == :all 
                data = df 
            elseif USstates != nothing
                data = filter(x -> x.province in string.(USstates), filter(x -> x.county in string.(UScounties), df))
            else
                data = filter(x -> x.county in string.(UScounties), df)  #UScounty is a vector of Symbols
            end
            
        else
            error("COVID-19 Totals are only available for all the countries ('countries = :all') and for US counties 
                (all counties 'counties = :all', or a subset of them 'counties = (:countie1, :countie2, ...))'.")
        end
    
        return data
   
    catch e
        print(e)        
    end
end


function opendisease_JHUCSSE(data_type::Val{:accumulated}; lastdays = nothing, countries = nothing, provinces = nothing, USstates = nothing, UScounties = nothing)   
    """
    'countries': An array of country names in iso2, iso3, or country ID code format.
    'provinces': An array of 'Symbol'. It contains provinces names spelled correctly separated by ','.
    """
    
    try
        base_jhucsse = "https://disease.sh/v3/covid-19/"
        base_url = assemble_base_url(base_jhucsse, data_type = data_type)
        params = Dict("lastdays" => lastdays)
        countries == :all ? (url = assemble_url_jhucsse(base_url, data_type, countries = countries)) : error("Accumulated data is olnly available for all the countries")
        get_data = HTTP.get(url, query = params)
        data = DataFrame(JSON3.read(get_data.body))
        #data = assemble_df_jhucsse(df)
            
        return data
   
    catch e
        print(e)        
    end
end


function opendisease_JHUCSSE(data_type::Val{:timeseries}; lastdays = nothing, countries = nothing, provinces = nothing, USstates = nothing, UScounties = nothing)
    """
    'countries': An array of country names in iso2, iso3, or country ID code format.
    'provinces': An array of 'Symbol'. It contains provinces names spelled correctly separated by ','.
    """
    
    try
        base_jhucsse = "https://disease.sh/v3/covid-19/"
        base_url = assemble_base_url(base_jhucsse, data_type = data_type)
        params = Dict("lastdays" => lastdays)
       
        if (countries != nothing && provinces == nothing)
            if countries == :all
                url = assemble_url_jhucsse(base_url, data_type, countries = countries)
                get_data = HTTP.get(url, query = params)
                df  = DataFrame(JSON3.read(get_data.body))
                data = assemble_df_jhucsse(df, countries = countries)
                #timeline = vcat(DataFrame.(df[:, :timeline])...)
                #data = hcat(select(df, Not(:timeline)), timeline)
                
            elseif countries != :all
                url = assemble_url_jhucsse(base_url, data_type, countries = countries)
                get_data = HTTP.get(url, query = params)
                length(countries) == 1 ? (df = DataFrame(JSON3.read(get_data.body))) : (df = vcat(DataFrame.(JSON3.read(get_data.body))...))
                data = assemble_df_jhucsse(df, countries = countries)
                #timeline = vcat(DataFrame.(df[:, :timeline])...)
                #data = hcat(select(df, Not(:timeline)), timeline)
            else
                error("'countries' must be ':all' or a vector of 'Symbol's with the name of the countries of interest")
            end   
        
        elseif provinces != nothing
            #provinces names must not be capitalized and the words must be separated by one space. Example: "australian capital territory"
            url = assemble_url_jhucsse(base_url, data_type, countries = countries, provinces = provinces)
            get_data = HTTP.get(url, query = params)
            df = DataFrame(JSON3.read(get_data.body))
            data = assemble_df_jhucsse(df, countries = countries)
            #timeline = vcat(DataFrame.(df[:, :timeline])...)
            #data = hcat(select(df, Not(:timeline)), timeline)
            #data = filter(x -> x.province in pronvinces, df
        elseif UScounties == :all
            url = assemble_url_jhucsse(base_url, data_type, UScounties = UScounties)
            get_data = HTTP.get(url, query = params)
            data = DataFrame(USstate = JSON3.read(get_data.body))
            #USstates == nothing ? data = filter(x -> x.county in string.(UScounties), df) : data = df
        
        elseif USstates != nothing
            url = assemble_url_jhucsse(base_url, data_type, USstates = USstates, UScounties = UScounties)
            get_data = HTTP.get(url, query = params)
            raw_df = DataFrame(JSON3.read(get_data.body))
            df = assemble_df_jhucsse(raw_df, USstates = USstates)
            #timeline = vcat(DataFrame.(raw_df[:, :timeline])...)
            #df = hcat(select(raw_df, Not(:timeline)), timeline)
            UScounties != nothing ? data = filter(x -> x.county in string.(UScounties), df) : data = df
        else
            nothing
        end
                                     
        return data
   
    catch e
        print(e) 
    end
end


function opendisease_JHUCSSE(data_type::Symbol; kwargs...)
    """
    'countries': An array of country names in iso2, iso3, or country ID code format.
    'provinces': An array of 'Symbol'. It contains provinces names spelled correctly separated by ','.
    """
    try
        opendisease_JHUCSSE(Val(data_type); kwargs...)
    catch e
        error("It was not possible to request the data as indicated. Please check if following requirements 
                are met in your data request: \n
                - COVID-19 Totals are only available for all the countries ('countries = :all') and for US counties 
                (all counties 'counties = :all', or a subset of them 'counties = (:countie1, :countie2, ...))'. \n
                - 'lastdays' only applies for time-series data, and it must be always specified. \n 
                - 'countries' must be a vesctor of countries' IDs, iso2 or iso3 \n")
         # for Time series data, the number of days (lastdays) must be specified \n
         # lastdays only applies for time series \n
    end 
end

In [235]:
opendisease_JHUCSSE(:timeseries, lastdays = 2, UScounties = :all);

In [236]:
data = opendisease_JHUCSSE(:timeseries, lastdays = 2, UScounties = [:dupage, :brown, :knox], USstates = [:illinois])

,province,county,cases,deaths
,String,String,Object…,Object…
1,illinois,brown,"{\n ""2/5/22"": 2319,\n ""2/6/22"": 2319\n}","{\n ""2/5/22"": 13,\n ""2/6/22"": 13\n}"
2,illinois,dupage,"{\n ""2/5/22"": 200020,\n ""2/6/22"": 200020\n}","{\n ""2/5/22"": 1611,\n ""2/6/22"": 1611\n}"
3,illinois,knox,"{\n ""2/5/22"": 12099,\n ""2/6/22"": 12099\n}","{\n ""2/5/22"": 199,\n ""2/6/22"": 199\n}"


In [237]:
opendisease_JHUCSSE(:timeseries, lastdays = 10, UScounties = [:dupage], USstates = [:illinois])

,province,county,cases
,String,String,Object…
1,illinois,dupage,"{\n ""1/28/22"": 196061,\n ""1/29/22"": 196061,\n ""1/30/22"": 196061,\n ""1/31/22"": 197510,\n ""2/1/22"": 198028,\n ""2/2/22"": 198539,\n ""2/3/22"": 198539,\n ""2/4/22"": 200020,\n ""2/5/22"": 200020,\n ""2/6/22"": 200020\n}"


In [218]:
opendisease_JHUCSSE(:accumulated; lastdays = 2, countries = :all)

,cases,deaths
,Object…,Object…
1,"{\n ""2/4/22"": 391129461,\n ""2/5/22"": 393217243\n}","{\n ""2/4/22"": 5726321,\n ""2/5/22"": 5734396\n}"


In [219]:
opendisease_JHUCSSE(:timeseries; lastdays = 2, countries = :all)

DimensionMismatch("Length of `row` does not match `DataFrame` column count.")

In [220]:
opendisease_JHUCSSE(:timeseries, lastdays = 2, countries = [:MX])

DimensionMismatch("Length of `row` does not match `DataFrame` column count.")

In [221]:
opendisease_JHUCSSE(:timeseries, lastdays = 2, countries = [:MX, :AU])

DimensionMismatch("Length of `row` does not match `DataFrame` column count.")

In [222]:
opendisease_JHUCSSE(:timeseries, lastdays = 2, countries = [:AU], provinces = [:queensland])

DimensionMismatch("Length of `row` does not match `DataFrame` column count.")

In [223]:
opendisease_JHUCSSE(:timeseries, lastdays = 2, countries = [:AU], provinces = [:queensland, :tasmania])

DimensionMismatch("Length of `row` does not match `DataFrame` column count.")

In [224]:
data1 = opendisease_JHUCSSE(:timeseries, countries = :all, lastdays = 2)

DimensionMismatch("Length of `row` does not match `DataFrame` column count.")

In [229]:
opendisease_JHUCSSE(:totals, countries = :all);

In [231]:
opendisease_JHUCSSE(:totals, UScounties = :all);

In [227]:
opendisease_JHUCSSE(:totals, UScounties = [:DuPage, :Brown, :Clark], USstates = [:Illinois, :Idaho]);

In [185]:
data1 = opendisease_JHUCSSE(:timeseries, countries = [:US], lastdays = "all")

DimensionMismatch("Length of `row` does not match `DataFrame` column count.")

In [162]:
date_arr = String.(keys(data1.cases[1]))

746-element Vector{String}:
 "1/22/20"
 "1/23/20"
 "1/24/20"
 "1/25/20"
 "1/26/20"
 "1/27/20"
 "1/28/20"
 "1/29/20"
 "1/30/20"
 "1/31/20"
 "2/1/20"
 "2/2/20"
 "2/3/20"
 ⋮
 "1/25/22"
 "1/26/22"
 "1/27/22"
 "1/28/22"
 "1/29/22"
 "1/30/22"
 "1/31/22"
 "2/1/22"
 "2/2/22"
 "2/3/22"
 "2/4/22"
 "2/5/22"

In [163]:
dict = [Dict(:country => country, :province => province, :date => date, :cases => data1.cases[1][date], 
        :deaths => data1.cases[1][date], :revovered => data1.recovered[1][date]) for (country, province) in eachrow(data1) for date in date_arr]

746-element Vector{Dict{Symbol, Any}}:
 Dict(:country => "USA", :cases => 1, :deaths => 1, :revovered => 0, :date => "1/22/20", :province => "mainland")
 Dict(:country => "USA", :cases => 1, :deaths => 1, :revovered => 0, :date => "1/23/20", :province => "mainland")
 Dict(:country => "USA", :cases => 2, :deaths => 2, :revovered => 0, :date => "1/24/20", :province => "mainland")
 Dict(:country => "USA", :cases => 2, :deaths => 2, :revovered => 0, :date => "1/25/20", :province => "mainland")
 Dict(:country => "USA", :cases => 5, :deaths => 5, :revovered => 0, :date => "1/26/20", :province => "mainland")
 Dict(:country => "USA", :cases => 5, :deaths => 5, :revovered => 0, :date => "1/27/20", :province => "mainland")
 Dict(:country => "USA", :cases => 5, :deaths => 5, :revovered => 0, :date => "1/28/20", :province => "mainland")
 Dict(:country => "USA", :cases => 6, :deaths => 6, :revovered => 0, :date => "1/29/20", :province => "mainland")
 Dict(:country => "USA", :cases => 6, :deaths => 

In [164]:
DataFrame(dict)

,country,cases,deaths,revovered,date,province
,String,Int64,Int64,Int64,String,String
1,USA,1,1,0,1/22/20,mainland
2,USA,1,1,0,1/23/20,mainland
3,USA,2,2,0,1/24/20,mainland
4,USA,2,2,0,1/25/20,mainland
5,USA,5,5,0,1/26/20,mainland
6,USA,5,5,0,1/27/20,mainland
7,USA,5,5,0,1/28/20,mainland
8,USA,6,6,0,1/29/20,mainland
9,USA,6,6,0,1/30/20,mainland


In [ ]:
data.cases

In [160]:
date_arr = String.(keys(data.cases[1]))
dict = [Dict(:cases => data.cases[1][date], :deaths => data.deaths[1][date], :recovered => data.recovered[1][date]) for date in date_arr]

746-element Vector{Dict{Symbol, Int64}}:
 Dict(:recovered => 0, :cases => 1, :deaths => 0)
 Dict(:recovered => 0, :cases => 1, :deaths => 0)
 Dict(:recovered => 0, :cases => 2, :deaths => 0)
 Dict(:recovered => 0, :cases => 2, :deaths => 0)
 Dict(:recovered => 0, :cases => 5, :deaths => 0)
 Dict(:recovered => 0, :cases => 5, :deaths => 0)
 Dict(:recovered => 0, :cases => 5, :deaths => 0)
 Dict(:recovered => 0, :cases => 6, :deaths => 0)
 Dict(:recovered => 0, :cases => 6, :deaths => 0)
 Dict(:recovered => 0, :cases => 8, :deaths => 0)
 Dict(:recovered => 0, :cases => 8, :deaths => 0)
 Dict(:recovered => 0, :cases => 8, :deaths => 0)
 Dict(:recovered => 0, :cases => 11, :deaths => 0)
 ⋮
 Dict(:recovered => 0, :cases => 72357997, :deaths => 873639)
 Dict(:recovered => 0, :cases => 73009708, :deaths => 877707)
 Dict(:recovered => 0, :cases => 73497946, :deaths => 880094)
 Dict(:recovered => 0, :cases => 74099283, :deaths => 883982)
 Dict(:recovered => 0, :cases => 74268633, :deaths => 885

In [ ]:
[Dict(:Date => String.(keys(data.cases[1])), :cases => values(data.cases[1])) for (dates, cases) #:deaths => values(data.cases[1]), :recovered => values(data.recovered[1]))

In [ ]:
cases = DataFrame([Dict(:date => dates, :cases => cases) for (dates, cases) in data.cases[1]])
deaths = DataFrame([Dict(:date => dates, :deaths => deaths) for (dates, deaths) in data.deaths[1]])
recovered = DataFrame([Dict(:date => dates, :recovered => recovered) for (dates, recovered) in data.recovered[1]])
df = innerjoin(cases, deaths, recovered, on = :date)

In [ ]:
Dict[ keys(data.cases[1]) == keys(data.deaths[1])

In [ ]:
DataFrame(data.cases)

In [ ]:
date_format = Dates.DateFormat("m/d/Y")

In [ ]:
String.(keys(cases))

In [ ]:
parse.(Date, String.(keys(cases)), date_format) .+ Year(2000)